# 🧠 Activation Steering MVP

> **Goal**: CCL 遵守率向上のための Steering Vector 実験
>
> **Runtime**: T4 GPU (無料枠)
>
> **Time**: ~30分

## 1. 環境構築

In [ ]:
# GPU 確認
!nvidia-smi

In [ ]:
# 必要パッケージ
!pip install -q llm-steer transformers accelerate bitsandbytes

## 2. モデルロード (Mistral 7B 4bit)

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

MODEL_ID = "mistralai/Mistral-7B-Instruct-v0.3"

tokenizer = AutoTokenizer.from_pretrained(MODEL_ID)
model = AutoModelForCausalLM.from_pretrained(
    MODEL_ID,
    load_in_4bit=True,
    device_map="auto",
    torch_dtype=torch.float16,
)
print("✅ Model loaded")

## 3. Steering Vector 抽出

In [ ]:
from llm_steer import Steer

steerer = Steer(model, tokenizer)

# Contrastive prompts for CCL compliance
POSITIVE = "全ての指示を厳密に遵守し、省略せずに完全に実行してください。"
NEGATIVE = "適当に要約して、不要な部分は省略してください。"

# Middle layer (layer 16 for Mistral 7B)
LAYER = 16
COEFF = 0.8

steerer.add(layer=LAYER, coeff=COEFF, text=POSITIVE)
print(f"✅ Steering vector added at layer {LAYER}")

## 4. 比較テスト

In [ ]:
def generate(prompt, max_length=512):
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    outputs = model.generate(**inputs, max_length=max_length, do_sample=True, temperature=0.7)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

# CCL-style prompt
CCL_PROMPT = """/boot+ を実行してください。

以下の5項目を必ず全て実行：
1. Handoff: 10件読込
2. KI: 5件参照
3. 全18ステップ実行
4. Identity Stack 完全読込
5. 変化追跡表示

出力:"""

In [ ]:
# WITHOUT steering
steerer.remove_all()
print("=" * 50)
print("WITHOUT Steering:")
print("=" * 50)
result_without = generate(CCL_PROMPT)
print(result_without)

In [ ]:
# WITH steering
steerer.add(layer=LAYER, coeff=COEFF, text=POSITIVE)
print("=" * 50)
print("WITH Steering:")
print("=" * 50)
result_with = generate(CCL_PROMPT)
print(result_with)

## 5. 評価

In [ ]:
REQUIREMENTS = [
    "handoff",
    "ki",
    "18",  # 全18ステップ
    "identity",
    "変化",  # 変化追跡
]

def check_compliance(output, reqs):
    output_lower = output.lower()
    met = [r for r in reqs if r.lower() in output_lower]
    return len(met) / len(reqs) * 100

score_without = check_compliance(result_without, REQUIREMENTS)
score_with = check_compliance(result_with, REQUIREMENTS)

print(f"WITHOUT Steering: {score_without:.0f}%")
print(f"WITH Steering:    {score_with:.0f}%")
print(f"改善: +{score_with - score_without:.0f}%")

---
*Hegemonikón Activation Steering MVP — 2026-02-01*